In [1]:
# pip install seaborn

In [2]:
%matplotlib qt5
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scyint
import pandas as pd
from mathphys import constants
from mathphys.functions import save_pickle, load_pickle
from mathphys.beam_optics import beam_rigidity


In [3]:
import seaborn

In [4]:
import touschek_pack.functions as tousfunc

In [5]:
import touschek_pack.Classes as tousclass

In [6]:
# eu preciso lembrar de depois acrescentar uma função que faça o trakcing nao tornando explicito a diferenciação entre desvios de energia 
# positivos e negativos para contabilizar de maneira correta a taxa de espalhamento touschek porque na biblioteca lifetime a taxa de espalhamento
# calculada esta implementada com base nos espalhamentos tanto negativos como positivos

In [7]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)

tousan = tousclass.Tous_analysis(acc)
spos = pyaccel.lattice.find_spos(acc, indices='closed')


In [8]:
# indices = pyaccel.lattice.find_indices(acc, 'fam_name', 'B2')
fam_data = pymodels.si.get_family_data(acc)
fam_data['BC']['index']

[[147,
  148,
  149,
  151,
  152,
  153,
  155,
  156,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  179,
  180,
  182,
  183,
  184,
  186,
  187,
  188],
 [474,
  475,
  476,
  478,
  479,
  480,
  482,
  483,
  485,
  486,
  487,
  488,
  489,
  490,
  491,
  492,
  493,
  496,
  497,
  498,
  499,
  500,
  501,
  502,
  503,
  504,
  506,
  507,
  509,
  510,
  511,
  513,
  514,
  515],
 [799,
  800,
  801,
  803,
  804,
  805,
  807,
  808,
  810,
  811,
  812,
  813,
  814,
  815,
  816,
  817,
  818,
  821,
  822,
  823,
  824,
  825,
  826,
  827,
  828,
  829,
  831,
  832,
  834,
  835,
  836,
  838,
  839,
  840],
 [1125,
  1126,
  1127,
  1129,
  1130,
  1131,
  1133,
  1134,
  1136,
  1137,
  1138,
  1139,
  1140,
  1141,
  1142,
  1143,
  1144,
  1147,
  1148,
  1149,
  1150,
  1151,
  1152,
  1153,
  1154,
  1155,
  1157,
  1158,
  1160,
  1161,
  1162,
  1164,
  1165,
  1166],
 [1447,
  

In [9]:
s = spos[474:515]
tousan.plot_scat_table(s, n_c=1, n_r=1)

In [ ]:
ltime = pyaccel.lifetime.Lifetime(acc)
tous_rate = ltime.touschek_data['rate']
np.where(tous_rate <= 1e-6)[0].shape